<a href="https://colab.research.google.com/github/rtldr/DeepDecompiler/blob/master/DeepDecompiler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
Test commit